# 用于金融应用的Claude Skills

使用Claude的Excel、PowerPoint和PDF技能构建真实的金融仪表板、投资组合分析和自动化报告工作流。

> **💡 实际影响：** 这些正是驱动**[Claude创建文件](https://www.anthropic.com/news/create-files)**的技能，使Claude能够在界面中直接创建专业金融文档。

**您将学到：**
- 在Excel中创建包含公式和图表的综合金融模型
- 从财务数据生成执行演示
- 构建包含风险指标的投资组合分析工具
- 自动化多格式报告管道

## 目录

1. [设置与数据加载](#setup)
2. [用例1：金融仪表板创建](#financial-dashboard)
   - [Excel金融模型](#excel-model)
   - [执行PowerPoint](#executive-ppt)
   - [PDF金融报告](#pdf-report)
3. [用例2：投资组合分析工作流](#portfolio-analysis)
   - [投资组合分析Excel](#portfolio-excel)
   - [投资委员会演示](#investment-deck)
4. [用例3：自动化报告管道](#reporting-pipeline)

## 先决条件

本notebook假设您已完成**Notebook 1：Skills入门**。

如果您尚未完成：
1. 首先完成Notebook 1中的设置
2. 使用测试单元格验证您的环境
3. 确保您可以创建和下载文件

**必需：**
- 配置的Anthropic API密钥
- 从whl安装的SDK版本0.69.0
- 激活的虚拟环境

## 1. 设置与数据加载 {#setup}

让我们首先导入依赖项并加载我们将在此notebook中使用的数据。

In [ ]:
# 标准导入
import json
import os
import sys
from pathlib import Path

import pandas as pd

# 添加父目录用于导入
sys.path.insert(0, str(Path.cwd().parent))

# Anthropic SDK
from anthropic import Anthropic
from dotenv import load_dotenv

# 我们的工具
from file_utils import (
    download_all_files,
    print_download_summary,
)

# 加载环境
load_dotenv(Path.cwd().parent / ".env")

# 配置
API_KEY = os.getenv("ANTHROPIC_API_KEY")
MODEL = "claude-sonnet-4-5"

if not API_KEY:
    raise ValueError("未找到ANTHROPIC_API_KEY。请配置您的.env文件。")

# 初始化客户端
client = Anthropic(api_key=API_KEY)

# 设置目录
OUTPUT_DIR = Path.cwd().parent / "outputs" / "financial"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

DATA_DIR = Path.cwd().parent / "sample_data"

print("✓ 环境已配置")
print(f"✓ 输出目录：{OUTPUT_DIR}")
print(f"✓ 数据目录：{DATA_DIR}")

### 加载金融数据

我们拥有代表公司财务状况不同方面的四个数据集：

In [ ]:
# 加载财务报表
financial_statements = pd.read_csv(DATA_DIR / "financial_statements.csv")
print("📊 财务报表概览：")
print(f"   形状：{financial_statements.shape}")
print(f"   类别：{len(financial_statements['Category'].unique())}个财务指标")
print(f"   季度：{list(financial_statements.columns[1:5])}")
print()

# 显示示例数据
print("示例数据（前5行）：")
financial_statements.head()

In [ ]:
# 加载投资组合持仓
with open(DATA_DIR / "portfolio_holdings.json") as f:
    portfolio_data = json.load(f)

print("💼 投资组合概览：")
print(f"   投资组合：{portfolio_data['portfolio_name']}")
print(f"   总价值：${portfolio_data['total_value']:,.2f}")
print(f"   持仓：{len(portfolio_data['holdings'])}只股票")
print(f"   现金头寸：${portfolio_data['cash_position']['amount']:,.2f}")
print(f"   总回报：{portfolio_data['performance_metrics']['total_return_percent']:.1f}%")
print()

# 将持仓转换为DataFrame以便操作
portfolio_df = pd.DataFrame(portfolio_data["holdings"])
print("按价值计算的前5大持仓：")
portfolio_df.nlargest(5, "market_value")[["ticker", "name", "market_value", "unrealized_gain"]]

In [ ]:
# 加载季度指标
with open(DATA_DIR / "quarterly_metrics.json") as f:
    quarterly_metrics = json.load(f)

print("📈 季度指标概览：")
print(f"   可用季度：{len(quarterly_metrics['quarters'])}个")
print(f"   每季度指标：{len(quarterly_metrics['quarters'][0])}个KPI")
print()

# 显示最新季度指标
latest_quarter = quarterly_metrics["quarters"][-1]
print(f"最新季度（{latest_quarter['quarter']}）：")
for key, value in latest_quarter.items():
    if key != "quarter" and isinstance(value, int | float):
        if "revenue" in key.lower() or "cost" in key.lower():
            print(f"   {key.replace('_', ' ').title()}：${value:,.0f}")
        elif "percent" in key.lower() or "margin" in key.lower() or "rate" in key.lower():
            print(f"   {key.replace('_', ' ').title()}：{value:.1f}%")
        else:
            print(f"   {key.replace('_', ' ').title()}：{value:,.0f}")

### 辅助函数

让我们为此notebook定义一些辅助函数：

In [ ]:
def create_skills_message(client, prompt, skills, prefix="", show_token_usage=True):
    """
    用于创建带有Skills的消息的辅助函数。

    参数:
        client: Anthropic客户端
        prompt: 用户提示
        skills: 技能字典列表 [{"type": "anthropic", "skill_id": "xlsx", "version": "latest"}]
        prefix: 下载文件的前缀
        show_token_usage: 是否打印令牌使用情况

    返回:
        (响应, 下载结果)的元组
    """
    response = client.beta.messages.create(
        model=MODEL,
        max_tokens=4096,
        container={"skills": skills},
        tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
        messages=[{"role": "user", "content": prompt}],
        betas=[
            "code-execution-2025-08-25",
            "files-api-2025-04-14",
            "skills-2025-10-02",
        ],
    )

    if show_token_usage:
        print(
            f"\n📊 令牌使用情况：{response.usage.input_tokens} 输入，{response.usage.output_tokens} 输出"
        )

    # 下载文件
    results = download_all_files(client, response, output_dir=str(OUTPUT_DIR), prefix=prefix)

    return response, results


def format_financial_value(value, is_currency=True, decimals=0):
    """格式化财务值以便显示。"""
    if is_currency:
        return f"${value:,.{decimals}f}"
    else:
        return f"{value:,.{decimals}f}"


print("✓ 辅助函数已定义")

## 2. 用例1：金融仪表板创建 {#financial-dashboard}

现在我们已经加载数据并定义了辅助函数，让我们深入第一个实际用例：创建综合金融仪表板。我们将首先生成多工作表Excel工作簿，自动包含公式、格式和图表。

### 2.1 Excel金融模型 {#excel-model}

我们将创建一个包含以下内容的金融仪表板：
- 损益表与同比比较
- 资产负债表分析
- 现金流跟踪
- 带可视化的KPI仪表板

这演示了Claude的技能如何处理通常需要数小时手动工作的复杂Excel生成任务。

In [ ]:
# 创建金融仪表板Excel
print("正在创建金融仪表板Excel文件...")
print("这创建了一个针对Skills API优化的2工作表仪表板。")
print("\n⏱️ 生成时间：1-2分钟\n")

# 准备财务数据
fs_data = financial_statements.to_dict("records")
quarters_2024 = ["Q1_2024", "Q2_2024", "Q3_2024", "Q4_2024"]

# 提取关键财务指标
revenue_by_quarter = {
    "Q1 2024": financial_statements[financial_statements["Category"] == "Revenue"][
        "Q1_2024"
    ].values[0],
    "Q2 2024": financial_statements[financial_statements["Category"] == "Revenue"][
        "Q2_2024"
    ].values[0],
    "Q3 2024": financial_statements[financial_statements["Category"] == "Revenue"][
        "Q3_2024"
    ].values[0],
    "Q4 2024": financial_statements[financial_statements["Category"] == "Revenue"][
        "Q4_2024"
    ].values[0],
}

financial_dashboard_prompt = f"""
创建一个包含2个工作表的金融仪表板Excel工作簿：

工作表1 - "损益汇总"：
创建一个2024年季度的损益汇总表，包含以下行：
- 收入：{", ".join([f"Q{i + 1}：${v / 1000000:.1f}M" for i, v in enumerate(revenue_by_quarter.values())])}
- 毛利润：使用数据中的值
- 营业收入：使用数据中的值
- 净利润：使用数据中的值
- 添加包含SUM公式的合计列
- 添加显示利润率（净利润/收入）的行
- 应用货币格式和粗体标题
- 添加显示季度收入的简单柱状图

工作表2 - "关键指标"：
创建包含以下内容的指标仪表板：
- 2024年总收入：所有季度的SUM
- 平均季度收入：AVERAGE公式
- Q4 vs Q1增长率：百分比增长
- 最佳季度：MAX公式识别
- Q4营业利润率：从数据计算
- 相比2023年的同比增长

应用专业格式，包括边框、粗体标题和货币格式。
"""

# 创建Excel金融仪表板
excel_response, excel_results = create_skills_message(
    client,
    financial_dashboard_prompt,
    [{"type": "anthropic", "skill_id": "xlsx", "version": "latest"}],
    prefix="financial_dashboard_",
)

print("\n" + "=" * 60)
print_download_summary(excel_results)

if len(excel_results) > 0 and excel_results[0]["success"]:
    print("\n✅ 金融仪表板Excel创建成功！")

### 💡 Excel生成最佳实践

基于我们的测试，以下是使用Skills创建Excel文件的最佳方法：

**推荐方法：**
- 每个工作簿**2-3个工作表**可靠且生成快速
- **专注于每个工作表**的具体目的（例如损益表、指标、图表）
- **增量增加复杂性** - 从简单开始，然后增强

**对于复杂仪表板：**
1. **创建多个专注的文件**而不是一个复杂文件
   - 示例：`financial_pnl.xlsx`、`balance_sheet.xlsx`、`kpi_dashboard.xlsx`
2. **使用管道模式**顺序创建和增强文件
3. **如需要，使用pandas或openpyxl以编程方式合并文件**

**性能提示：**
- 简单的2工作表仪表板：约1-2分钟
- PowerPoint和PDF生成：非常可靠，适用于复杂内容
- 令牌使用：结构化数据（JSON/CSV）比散文更高效

### 2.2 执行PowerPoint {#executive-ppt}

现在我们的财务数据已在Excel中组织，让我们创建一个执行演示，总结关键见解。这演示了技能如何生成带有图表、格式化文本和多个幻灯片的专业PowerPoint演示 - 非常适合董事会会议或投资者更新。

演示将包括：
- 2024年第四季度业绩亮点
- 财务指标与同比比较
- 盈利趋势与可视化
- 关键要点和展望

In [ ]:
print("正在从财务指标创建执行演示...")
print("\n⏱️ 生成时间：1-2分钟\n")

# 计算演示的一些关键指标
q4_2024_revenue = 14500000
q4_2023_revenue = 12300000
yoy_growth = (q4_2024_revenue - q4_2023_revenue) / q4_2023_revenue * 100

q4_2024_net_income = 1878750
q4_2023_net_income = 1209000
net_income_growth = (q4_2024_net_income - q4_2023_net_income) / q4_2023_net_income * 100

executive_ppt_prompt = f"""
创建一个包含4页的2024年第四季度财务结果执行演示：

第1页 - 标题：
- 标题："2024年第四季度财务结果"
- 副标题："执行摘要 - Acme公司"
- 日期：2025年1月

第2页 - 财务亮点：
- 标题："2024年第四季度业绩亮点"
- 创建两列布局：
  左侧 - 关键指标：
  • 收入：$14.5M（+{yoy_growth:.1f}%同比增长）
  • 净利润：$1.88M（+{net_income_growth:.1f}%同比增长）
  • 营业利润率：17.9%（上升2.9个百分点）
  • 经营现金流：$2.85M

  右侧 - 显示季度收入的柱状图：
  2024年第一季度：$12.5M
  2024年第二季度：$13.2M
  2024年第三季度：$13.8M
  2024年第四季度：$14.5M

第3页 - 盈利趋势：
- 标题："利润率扩张与盈利能力"
- 添加显示按季度划分的净利润率的折线图：
  2024年第一季度：11.4%
  2024年第二季度：11.8%
  2024年第三季度：12.4%
  2024年第四季度：13.0%
- 在下方添加要点：
  • 2024年全年利润率持续扩张
  • 经营杠杆推动盈利能力
  • 成本优化措施产生效果

第4页 - 关键要点：
- 标题："关键要点与展望"
- 要点：
  ✓ 创纪录的第四季度收入$14.5M
  ✓ 17.9%的收入同比增长
  ✓ 净利润同比增长55%
  ✓ 强劲的现金生成：$2.85M经营现金流
  ✓ 为2025年持续增长做好准备

使用专业企业设计：
- 深蓝色（#003366）用于标题
- 简洁、现代的布局
- 数据驱动的可视化
"""

# 创建执行演示
ppt_response, ppt_results = create_skills_message(
    client,
    executive_ppt_prompt,
    [{"type": "anthropic", "skill_id": "pptx", "version": "latest"}],
    prefix="executive_summary_",
)

print("\n" + "=" * 60)
print_download_summary(ppt_results)

if len(ppt_results) > 0 and ppt_results[0]["success"]:
    print("\n✅ 执行演示创建成功！")

## 3. 用例2：投资组合分析工作流 {#portfolio-analysis}

现在让我们将焦点从公司财务转向投资组合分析。在本节中，我们将演示如何使用之前加载的投资组合数据创建综合投资组合分析和投资委员会演示。

此工作流展示：
- Excel中的详细投资组合业绩分析
- 风险指标和行业分配可视化
- 专业投资委员会演示
- 数据驱动的再平衡建议

我们将首先创建包含投资组合分析的Excel工作簿，然后生成总结我们发现的投资委员会演示。

### 首先，让我们创建一个综合投资组合分析Excel工作簿

在创建投资委员会演示之前，我们需要详细分析我们的投资组合数据。这个Excel工作簿将作为我们投资建议的基础。

In [ ]:
print("正在创建投资组合分析Excel工作簿...")
print("这创建了一个针对Skills API优化的专注2工作表投资组合分析。")
print("\n⏱️ 生成时间：1-2分钟\n")

# 为提示准备投资组合数据
top_holdings = portfolio_df.nlargest(5, "market_value")
sector_allocation = portfolio_data["sector_allocation"]

portfolio_excel_prompt = f"""
创建一个包含2个工作表的投资组合分析Excel工作簿：

工作表1 - "投资组合概览"：
创建一个综合持仓和业绩表：

第1部分 - 持仓（工作表顶部）：
{portfolio_df[["ticker", "name", "shares", "current_price", "market_value", "unrealized_gain", "allocation_percent"]].head(10).to_string()}

第2部分 - 投资组合摘要：
- 投资组合总价值：${portfolio_data["total_value"]:,.2f}
- 总未实现收益：${portfolio_df["unrealized_gain"].sum():,.2f}
- 总回报率：{portfolio_data["performance_metrics"]["total_return_percent"]:.1f}%
- 年初至今回报率：{portfolio_data["performance_metrics"]["year_to_date_return"]:.1f}%
- 夏普比率：{portfolio_data["performance_metrics"]["sharpe_ratio"]:.2f}
- 投资组合Beta：{portfolio_data["performance_metrics"]["beta"]:.2f}

应用条件格式：绿色表示收益，红色表示损失。
添加显示前5大持仓价值的柱状图。

工作表2 - "行业分析与风险"：
创建行业分配和风险指标：

第1部分 - 行业分配：
{json.dumps(sector_allocation, indent=2)}
包含行业分配的饼图。

第2部分 - 关键风险指标：
- 投资组合Beta：{portfolio_data["performance_metrics"]["beta"]:.2f}
- 标准差：{portfolio_data["performance_metrics"]["standard_deviation"]:.1f}%
- 风险价值（95%）：$62,500
- 最大回撤：-12.3%
- 夏普比率：{portfolio_data["performance_metrics"]["sharpe_ratio"]:.2f}

第3部分 - 再平衡建议：
- 将技术行业从20%减少到18%
- 将医疗保健从8.7%增加到10%
- 保持当前多样化

应用专业格式，包括清晰的区域和标题。
"""

# 创建投资组合分析Excel
portfolio_response, portfolio_results = create_skills_message(
    client,
    portfolio_excel_prompt,
    [{"type": "anthropic", "skill_id": "xlsx", "version": "latest"}],
    prefix="portfolio_analysis_",
)

print("\n" + "=" * 60)
print_download_summary(portfolio_results)

if len(portfolio_results) > 0 and portfolio_results[0]["success"]:
    print("\n✅ 投资组合分析Excel创建成功！")

### 3.2 投资委员会演示 {#investment-deck}

随着我们详细的投资组合分析完成，现在让我们为投资委员会创建一个专业演示。此演示将把Excel分析中的关键见解提炼为适合决策者的简洁、视觉格式。

演示将涵盖：
- 包含关键指标的投资组合业绩摘要
- 资产配置和多元化分析
- 风险指标和风险调整回报
- 再平衡的战略建议

In [ ]:
print("正在创建投资委员会演示...")
print("\n⏱️ 生成时间：1-2分钟\n")

investment_deck_prompt = f"""
创建一个包含5页的投资委员会演示：

第1页 - 标题：
- 标题："投资组合审查 - 2024年第四季度"
- 副标题："{portfolio_data["portfolio_name"]}"
- 日期：2025年1月
- 投资组合价值：${portfolio_data["total_value"]:,.0f}

第2页 - 投资组合概览：
- 标题："投资组合业绩摘要"
- 两列布局：

  左侧 - 关键指标：
  • 总价值：${portfolio_data["total_value"]:,.0f}
  • 年初至今回报：+{portfolio_data["performance_metrics"]["year_to_date_return"]:.1f}%
  • 总回报：${portfolio_data["performance_metrics"]["total_return"]:,.0f}
  • 夏普比率：{portfolio_data["performance_metrics"]["sharpe_ratio"]:.2f}

  右侧 - 按价值显示前5大持仓的柱状图：
  {", ".join([f"{h['ticker']}: ${h['market_value']:,.0f}" for h in top_holdings.to_dict("records")])}

第3页 - 行业配置：
- 标题："资产配置与多元化"
- 饼图显示：
  技术：{sector_allocation["Technology"]:.1f}%
  金融：{sector_allocation["Financials"]:.1f}%
  医疗保健：{sector_allocation["Healthcare"]:.1f}%
  消费：{sector_allocation["Consumer Discretionary"] + sector_allocation["Consumer Staples"]:.1f}%
  固定收益：{sector_allocation["Bonds"]:.1f}%
  现金：{sector_allocation["Cash"]:.1f}%

第4页 - 风险分析：
- 标题："风险指标与分析"
- 内容：
  风险指标：
  • 投资组合Beta：{portfolio_data["performance_metrics"]["beta"]:.2f}（较低的市场风险）
  • 标准差：{portfolio_data["performance_metrics"]["standard_deviation"]:.1f}%
  • 最大回撤：-12.3%
  • 风险价值（95%）：$62,500

  风险调整业绩：
  • 夏普比率：{portfolio_data["performance_metrics"]["sharpe_ratio"]:.2f}（优秀）
  • Alpha生成：比基准+2.3%

第5页 - 建议：
- 标题："战略建议"
- 要点：
  ✓ 保持当前配置 - 良好多元化
  ✓ 考虑在技术行业获利了结（20% → 18%）
  ✓ 增加医疗保健配置（8.7% → 10%）
  ✓ 鉴于利率环境监控债券久期
  ✓ 季度重新平衡以保持目标

使用专业投资演示设计。
"""

# 创建投资委员会演示
investment_response, investment_results = create_skills_message(
    client,
    investment_deck_prompt,
    [{"type": "anthropic", "skill_id": "pptx", "version": "latest"}],
    prefix="investment_committee_",
)

print("\n" + "=" * 60)
print_download_summary(investment_results)
print("\n✅ 投资委员会演示创建成功！")

## 4. 用例3：自动化报告管道 {#reporting-pipeline}

到目前为止，我们已经为特定目的创建了个别文档。现在让我们演示在自动化工作流中将多个技能链接在一起的力量。这种管道模式对于生产系统至关重要，在这些系统中，您需要从同一数据源生成多个相关文档。

在此示例中，我们将创建一个完整的报告套件：
1. **在Excel中分析数据**并包含计算和图表
2. **在PowerPoint演示中总结见解**
3. **在正式PDF报告中记录过程**

这展示了技能如何协同工作，创建传统上需要多种工具和手动协调的综合报告解决方案。

**管道方法的关键优势：**
- 所有文档中数据一致
- 减少总生成时间
- 令牌使用优化
- 可扩展到多种报告类型

**⏱️ 预期总时间：** 完整管道2-3分钟

In [ ]:
print("🔄 启动自动化报告管道")
print("=" * 60)
print("这将创建一个完整的报告套件：")
print("1. Excel分析 → 2. PowerPoint摘要 → 3. PDF文档")
print("\n⏱️ 管道总时间：2-3分钟\n")

# 跟踪管道中的令牌使用情况
pipeline_tokens = {"input": 0, "output": 0}

# 步骤1：创建Excel分析
print("步骤1/3：创建包含季度指标的Excel分析...")

excel_pipeline_prompt = f"""
创建季度业务指标Excel文件：

工作表1 - "季度KPI"：
创建包含2024年这些季度指标的表格：
{
    json.dumps(
        [
            {
                k: v
                for k, v in q.items()
                if k in ["quarter", "revenue", "gross_margin", "customer_count", "churn_rate"]
            }
            for q in quarterly_metrics["quarters"]
        ],
        indent=2,
    )
}

添加：
- 环比增长计算
- 平均值和总计行
- 趋势的条件格式
- 显示收入趋势的折线图
- 显示客户数量的柱状图

工作表2 - "同比比较"：
比较所有指标的2024年第四季度与2023年第四季度。
计算百分比变化并突出显示改进。

专业格式，包括标题和边框。
"""

excel_response, excel_results = create_skills_message(
    client,
    excel_pipeline_prompt,
    [{"type": "anthropic", "skill_id": "xlsx", "version": "latest"}],
    prefix="pipeline_1_metrics_",
    show_token_usage=False,
)

pipeline_tokens["input"] += excel_response.usage.input_tokens
pipeline_tokens["output"] += excel_response.usage.output_tokens
print(
    f"✓ Excel已创建 - 令牌：{excel_response.usage.input_tokens} 输入，{excel_response.usage.output_tokens} 输出"
)

# 步骤2：创建PowerPoint摘要
print("\n步骤2/3：从指标创建PowerPoint摘要...")

ppt_pipeline_prompt = """
创建一个包含3页的季度指标摘要演示：

第1页：
- 标题："2024年第四季度指标摘要"
- 副标题："自动化报告管道演示"

第2页：
- 标题："关键绩效指标"
- 显示2024年第四季度指标：
  • 收入：$3.2M（+15%环比）
  • 客户：850（+8.9%环比）
  • 毛利率：72%
  • 流失率：2.8%（从3.5%改善）
- 添加比较第三季度与第四季度收入的简单柱状图

第3页：
- 标题："季度趋势分析"
- 显示收入增长第一季度至第四季度的折线图
- 关键见解要点：
  • 持续环比增长
  • 客户获取加速
  • 流失减少成功

简洁、数据导向的设计。
"""

ppt_response, ppt_results = create_skills_message(
    client,
    ppt_pipeline_prompt,
    [{"type": "anthropic", "skill_id": "pptx", "version": "latest"}],
    prefix="pipeline_2_summary_",
    show_token_usage=False,
)

pipeline_tokens["input"] += ppt_response.usage.input_tokens
pipeline_tokens["output"] += ppt_response.usage.output_tokens
print(
    f"✓ PowerPoint已创建 - 令牌：{ppt_response.usage.input_tokens} 输入，{ppt_response.usage.output_tokens} 输出"
)

# 步骤3：创建PDF文档
print("\n步骤3/3：创建PDF文档...")

pdf_pipeline_prompt = """
创建一个总结季度报告管道的PDF文档：

自动化报告管道
2024年第四季度结果文档

执行摘要
本文档总结了我们自动化报告管道生成的2024年第四季度业务指标。

关键指标
- 收入：$3.2M（15%环比增长）
- 客户群：850个活跃客户
- 毛利率：72%
- 流失率：2.8%（从3.5%改善）

管道组件
1. 数据处理：Excel中分析的季度指标
2. 可视化：PowerPoint中呈现的关键见解
3. 文档：PDF中生成的正式报告

自动化优势
• 报告时间减少90%
• 格式和质量一致
• 消除手动错误
• 可扩展到多种报告

下一步
- 扩展管道以包括预测分析
- 添加自动化电子邮件分发
- 实施实时数据源

生成时间：2025年1月
管道版本：1.0

格式为专业技术文档。
"""

pdf_response, pdf_results = create_skills_message(
    client,
    pdf_pipeline_prompt,
    [{"type": "anthropic", "skill_id": "pdf", "version": "latest"}],
    prefix="pipeline_3_documentation_",
    show_token_usage=False,
)

pipeline_tokens["input"] += pdf_response.usage.input_tokens
pipeline_tokens["output"] += pdf_response.usage.output_tokens
print(
    f"✓ PDF已创建 - 令牌：{pdf_response.usage.input_tokens} 输入，{pdf_response.usage.output_tokens} 输出"
)

# 管道摘要
print("\n" + "=" * 60)
print("🎯 管道完成！")
print("=" * 60)

print("\n📊 管道令牌使用摘要：")
print(f"   总输入令牌：{pipeline_tokens['input']:,}")
print(f"   总输出令牌：{pipeline_tokens['output']:,}")
print(f"   总令牌：{pipeline_tokens['input'] + pipeline_tokens['output']:,}")
print(f"   平均每个文档：{(pipeline_tokens['input'] + pipeline_tokens['output']) // 3:,}")

print("\n📁 生成的文件：")
all_results = excel_results + ppt_results + pdf_results
for i, result in enumerate(all_results, 1):
    if result["success"]:
        print(f"   {i}. {os.path.basename(result['output_path'])} ({result['size'] / 1024:.1f} KB)")

print("\n✅ 自动化报告管道执行成功！")
print("   所有三个文档都创建并在工作流中链接。")

## 总结与下一步

### 我们取得的成就

在本notebook中，您已经学会了如何：

✅ **金融仪表板创建**
- 构建带公式和图表的多工作表Excel模型
- 生成执行PowerPoint演示
- 创建专业PDF报告

✅ **投资组合分析**
- 开发投资组合分析工作簿
- 创建投资委员会演示
- 实施风险指标和再平衡工具

✅ **自动化管道**
- 链接多种文档格式
- 优化令牌使用
- 构建生产就绪模式

### 关键要点

1. **技能大幅简化金融文档创建** - 手动需要数小时的工作现在只需几分钟
2. **令牌效率极佳** - 技能使用的令牌比手动指令少约90%
3. **质量达到专业级** - 文档在业务环境中可直接使用
4. **自动化简单明了** - 管道模式支持复杂工作流

### 继续学习

📚 **下一步：[Notebook 3 - 自定义技能开发](03_skills_custom_development.ipynb)**
- 构建您自己的专业金融技能
- 创建公司特定的模板
- 实施高级自动化

### 尝试这些实验

1. **修改金融仪表板**以包含您自己的指标
2. **创建自定义投资组合**与不同的资产类别
3. **为您的特定报告需求构建管道**
4. **实验复杂性**以了解生成时间
5. **跟踪不同文档类型的令牌使用情况**

### 资源

- [Claude API文档](https://docs.anthropic.com/en/api/messages)
- [技能文档](https://docs.claude.com/en/docs/agents-and-tools/agent-skills/overview)
- [最佳实践](https://docs.claude.com/en/docs/agents-and-tools/agent-skills/best-practices)
- [文件API参考](https://docs.claude.com/en/api/files-content)